# Description

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

## Objective
In this challenge we try to predict whether a person aboard the Titanic would survive or not, based upon certain chareacteristics.

## Skills :
<ol>
<li> Basic Plotting Techniques in Seaborn </li>
<li> Methodology of Data Exploration </li>
<li> Treating Missing Data </li>
</ol>

### Machine Learning Algorithm :
<ul>
<li> Binary Classification/Logistic Regression </li>
</ul>

#  Code and Analysis Begins

First we put in all the required imports for the basic python libraries which are needed.

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Now we import our machine learning model for logistic regression from the scikit-learn library in python.

In [22]:
from sklearn.linear_model import LogisticRegression

## Getting the Data

Our data is saved in two files in the csv format:-
<ol>
<li> train.csv : This file contains the data for training our model </li>
<li> test.csv : This file has the data on which we will test our model and evaluate its accuracy </li>
</ol>

<b>We now import the data into pandas dataframes using the read_csv method</b>

In [23]:
titanic_train = pd.read_csv("train.csv")

In [24]:
titanic_test = pd.read_csv("test.csv")

## Getting to know the Data

In [27]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We have the following information and characteristics available for each of the passengers aboard the Titanic:-
<ol>
<li> PassengerId : Same as a serial number
<li> Survived : Classifying into 0= "not survived" and 1 = "Survived"
<li> PClass : The type of passenger class of ticket i.e. First, Second and Third
<li> Name : Name of the Passenger
<li> Sex : Gender of the Passenger
<li> Age : Age of the Passenger
<li> SibSp : Number of siblings + spouse(if any) on board
<li> Parch : Number of Parents + children (if on board)
<li> Ticket : Ticket Number
<li> Fare : The amount paid for ticket
<li> Cabin : The Cabin number of the passenger
<li> Embarked : The port from which the passenger embarked
</ol>
    